<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AB-Testing" data-toc-modified-id="AB-Testing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AB-Testing</a></span></li></ul></div>

# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

In [2]:
# librerias

import pandas as pd
import numpy as np

from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import norm, sem

import pylab as plt

import seaborn as sns

In [34]:
# datos
ab_test = pd.read_csv('data/cookie_cats.csv')

ab_test.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [4]:
# transformacion
ab_test.version.value_counts()      #   No hay errores entre los usuarios de cada versión de puerta

ab_test.retention_1.value_counts()  #   Tampoco en retention 1

ab_test.retention_7.value_counts()  #   Ni en retention 7



retention_7
False    73408
True     16781
Name: count, dtype: int64

### AB Testing 

In [5]:
# Muestreo RETENTION 1

control_ret1 = ab_test[ab_test.version=='gate_30']['retention_1']

trat_ret1 = ab_test[ab_test.version=='gate_40']['retention_1']

In [6]:
# Testeo RETENTION 1

impresiones = [control_ret1.shape[0], trat_ret1.shape[0]]  

conversiones = [sum(control_ret1), sum(trat_ret1)]  



z_score, p_value = proportions_ztest(conversiones, 
                                     nobs=impresiones)


(control_a, trata_a), (control_b, trata_b) = proportion_confint(conversiones, 
                                                                nobs=impresiones,
                                                                alpha=0.05)

In [7]:
print(f'z-score: {z_score:.2f}')

print(f'p-valor: {p_value:.3f}')

print(f'intervalo conf 95% para grupo control: [{control_a:.3f}, {control_b:.3f}]')

print(f'intervalo conf 95% para grupo tratamiento: [{trata_a:.3f}, {trata_b:.3f}]')

z-score: 1.78
p-valor: 0.074
intervalo conf 95% para grupo control: [0.444, 0.453]
intervalo conf 95% para grupo tratamiento: [0.438, 0.447]


Como el pvalor es mayor que 0.05, no podemos rechazar H0, por lo tanto, no existen diferencias significativas entre los usuarios que pagan en la puerta 30 o en la puerta 40 para la retención 1

In [8]:
# Muestreo RETENTION 7

control_ret7 = ab_test[ab_test.version=='gate_30']['retention_7']

trat_ret7 = ab_test[ab_test.version=='gate_40']['retention_7']

In [9]:
# Testeo RETENTION 7

impresiones = [control_ret7.shape[0], trat_ret7.shape[0]]  

conversiones = [sum(control_ret7), sum(trat_ret7)]  



z_score, p_value = proportions_ztest(conversiones, 
                                     nobs=impresiones)


(control_a, trata_a), (control_b, trata_b) = proportion_confint(conversiones, 
                                                                nobs=impresiones,
                                                                alpha=0.05)

In [10]:
print(f'z-score: {z_score:.2f}')

print(f'p-valor: {p_value:.3f}')

print(f'intervalo conf 95% para grupo control: [{control_a:.3f}, {control_b:.3f}]')

print(f'intervalo conf 95% para grupo tratamiento: [{trata_a:.3f}, {trata_b:.3f}]')

z-score: 3.16
p-valor: 0.002
intervalo conf 95% para grupo control: [0.187, 0.194]
intervalo conf 95% para grupo tratamiento: [0.178, 0.186]


In [33]:
#Comparación de medias


(0.4481879194630872, 0.5518120805369128)

In [12]:
retention_1= pd.crosstab(ab_test.version, ab_test.retention_1)

retention_1

retention_1,False,True
version,,
gate_30,24666,20034
gate_40,25370,20119


In [13]:
retention_7= pd.crosstab(ab_test.version, ab_test.retention_7)

retention_7

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


In [14]:
# muestreo
# GATE 30
imp_ctr_1, convs_ctr_1 = 24666+ 20034, 20034   # entrar, pagar
imp_ctr_7, convs_ctr_7 = 24666+ 20034, 8502   # entrar, pagar

# GATE 40
imp_trt_1, convs_trt_1 = 25370+ 20119, 20119   # entrar, pagar
imp_trt_7, convs_trt_7 = 25370+ 20119, 8279   # entrar, pagar


In [15]:
24666+ 20034

44700

In [16]:
# testeo para retention_1
impresiones_1 = [imp_ctr_1 , imp_trt_1]

conversiones_1 = [convs_ctr_1 , convs_ctr_7]

z_score, p_value = proportions_ztest(conversiones_1, 
                                     nobs=impresiones_1)

(control_a, trata_a), (control_b, trata_b) = proportion_confint(conversiones_1, 
                                                                nobs=impresiones_1,
                                                                alpha=0.05)

In [17]:
print(f'z-score: {z_score:.2f}')

print(f'p-valor: {p_value:.3f}')

print(f'intervalo conf 95% para grupo control: [{control_a:.3f}, {control_b:.3f}]')

print(f'intervalo conf 95% para grupo tratamiento: [{trata_a:.3f}, {trata_b:.3f}]')

z-score: 84.36
p-valor: 0.000
intervalo conf 95% para grupo control: [0.444, 0.453]
intervalo conf 95% para grupo tratamiento: [0.183, 0.190]


In [18]:
# testeo para retention_7



In [19]:
# conclusiones
# Explica detalladamente las conclusiones obtenidas de. tu testeo.